In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-hubertclass-test.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_hubertclass import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        names, hyps, txts, ans = solver.exec()
        dfs.append(pd.DataFrame(
            {'name': names,
             'hyps': hyps,
             'truth': txts,
             'ifcorrect': ans
            }))
        print(dfs[idx])
        print(names)
        print(hyps)
        print(txts)
        print(ans)
        del solver


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT5-1']
Mozillacv11Dataset CTT5-1 found wav data: 21
text len: 21
remove None, then wav data: 18, text len: 18
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 64, text len: 64
Setup ASR model and optimizer 
['54CTT']   0.010377069935202599   0.0   True
['78CTT']   0.09295611828565598   0.0   True
['20CTT']   0.6567965149879456   1.0   True
['74CTT']   0.18529048562049866   0.0   True
['67CTT']   0.556343674659729   1.0   True
['58CTT']   0.8055482506752014   1.0   True
['68CT

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTT5-4']
Mozillacv11Dataset CTT5-4 found wav data: 19
text len: 19
remove None, then wav data: 18, text len: 18
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 64, text len: 64
Setup ASR model and optimizer 
['44CTT']   0.06836926937103271   0.0   True
['21CTT']   0.9387480020523071   1.0   True
['16CTT']   0.9551706910133362   1.0   True
['32CTT']   0.8817943334579468   1.0   True
['79CTT']   0.02766905352473259   0.0   True
['12CTT']   0.989678144454956   1.0   True
['96CTT']   0.1940489113330841   0.0   True
['18CTT']   0.7932760715484619   1.0   True
['7CTT']   0.14814606308937073   0.0   True
['84CTT']   0.09812150895595

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])
    df.to_csv(f'./biclass_result/hubert9k-{idx+1}.csv', index=False)
alldf.to_csv(f'./biclass_result/hubert9k.csv', index=False) 

In [ ]:
print('asd')

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.



In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import numpy as np
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))
pred = np.array([0.0443, 0.0140, 0.3337, 0.0560, 0.0385, 0.5118, 0.1961, 0.1746, 0.0268, 0.2939, 0.0169, 0.6525, 0.3661, 0.1010, 0.0077, 0.0994, 0.2264, 0.6143, 0.0750, 0.4048, 0.2368])
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_true), np.array(pred), pos_label=1)
print('AUC: ',metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

 non-patient       0.80      0.92      0.86        13
     patient       0.83      0.62      0.71         8

    accuracy                           0.81        21
   macro avg       0.82      0.77      0.79        21
weighted avg       0.81      0.81      0.80        21

sensitivity:  0.625
specificity:  0.9230769230769231
AUC:  0.5288461538461539
